In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:

!gdown "https://drive.google.com/uc?id=1O4-mRTgHK4AntMNDdQmOP-jnxXs9agnQ&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1O4-mRTgHK4AntMNDdQmOP-jnxXs9agnQ&confirm=t
To: /kaggle/working/tennis_court_det_dataset.zip
100%|██████████████████████████████████████| 7.26G/7.26G [01:16<00:00, 95.2MB/s]


In [2]:
!unzip "tennis_court_det_dataset.zip"

Archive:  tennis_court_det_dataset.zip
   creating: data/
  inflating: data/data_train.json    
  inflating: data/data_val.json      
   creating: data/images/
  inflating: data/images/-0M6ixK7aIU_1050.png  
  inflating: data/images/-0M6ixK7aIU_1100.png  
  inflating: data/images/-0M6ixK7aIU_1150.png  
  inflating: data/images/-0M6ixK7aIU_1200.png  
  inflating: data/images/-0M6ixK7aIU_1250.png  
  inflating: data/images/-0M6ixK7aIU_1300.png  
  inflating: data/images/-0M6ixK7aIU_1350.png  
  inflating: data/images/-0M6ixK7aIU_1500.png  
  inflating: data/images/-0M6ixK7aIU_1550.png  
  inflating: data/images/-0M6ixK7aIU_1600.png  
  inflating: data/images/-0M6ixK7aIU_1650.png  
  inflating: data/images/-0M6ixK7aIU_1700.png  
  inflating: data/images/-0M6ixK7aIU_1900.png  
  inflating: data/images/-0M6ixK7aIU_1950.png  
  inflating: data/images/-0M6ixK7aIU_2000.png  
  inflating: data/images/-0M6ixK7aIU_2050.png  
  inflating: data/images/-0M6ixK7aIU_2100.png  
  inflating: data/images

## Importing Libraries

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Create torch dataset

In [5]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [6]:
train_dataset = KeypointsDataset("data/images","data/data_train.json")
val_dataset = KeypointsDataset("data/images","data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

## Using RESNET50 replacing its Last Layer

In [7]:
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 168MB/s] 


In [11]:
model = model.to(device)

## Train the model

In [12]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [13]:
epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14369.6181640625
Epoch 0, iter 10, loss: 14653.404296875
Epoch 0, iter 20, loss: 13685.98046875
Epoch 0, iter 30, loss: 14403.556640625
Epoch 0, iter 40, loss: 12476.8134765625
Epoch 0, iter 50, loss: 12861.44921875
Epoch 0, iter 60, loss: 12785.869140625
Epoch 0, iter 70, loss: 12308.7080078125
Epoch 0, iter 80, loss: 11261.32421875
Epoch 0, iter 90, loss: 11616.3330078125
Epoch 0, iter 100, loss: 11208.7724609375
Epoch 0, iter 110, loss: 10864.6904296875
Epoch 0, iter 120, loss: 10552.6982421875
Epoch 0, iter 130, loss: 9756.208984375
Epoch 0, iter 140, loss: 9757.1796875
Epoch 0, iter 150, loss: 9889.009765625
Epoch 0, iter 160, loss: 9435.0732421875
Epoch 0, iter 170, loss: 8665.9873046875
Epoch 0, iter 180, loss: 8177.986328125
Epoch 0, iter 190, loss: 7648.92333984375
Epoch 0, iter 200, loss: 8412.951171875
Epoch 0, iter 210, loss: 7895.19775390625
Epoch 0, iter 220, loss: 7503.5263671875
Epoch 0, iter 230, loss: 6695.82275390625
Epoch 0, iter 240, loss: 65

In [14]:
torch.save(model.state_dict(), "keypoints_model_50.pth")